In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
import os 
from tqdm import tqdm
from pandarallel import pandarallel
import warnings
warnings.filterwarnings('ignore')
pandarallel.initialize(progress_bar=True, nb_workers=4)
import re
import sklearn.preprocessing as preprocessing
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [2]:
data=pd.read_csv('Data/NGSIM_.csv')
data.head()

,Vehicle_ID,Frame_ID,Total_Frames,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_Length,v_Width,v_Class,v_Vel,v_Acc,Lane_ID,Preceding,Following,Space_Headway,Time_Headway
0,1,12,884,1113433136100,16.884,48.213,6042842.116,2133117.662,14.3,6.4,2,12.5,0.0,2,0,0,0.0,0.0
1,1,13,884,1113433136200,16.938,49.463,6042842.012,2133118.909,14.3,6.4,2,12.5,0.0,2,0,0,0.0,0.0
2,1,14,884,1113433136300,16.991,50.712,6042841.908,2133120.155,14.3,6.4,2,12.5,0.0,2,0,0,0.0,0.0
3,1,15,884,1113433136400,17.045,51.963,6042841.805,2133121.402,14.3,6.4,2,12.5,0.0,2,0,0,0.0,0.0
4,1,16,884,1113433136500,17.098,53.213,6042841.701,2133122.649,14.3,6.4,2,12.5,0.0,2,0,0,0.0,0.0


# Preparation de data
### recherche des voisins

In [ ]:
def get_neighbors(row,data,max_dist=70,max_width=5.4):
    import pandas as pd
    
    #apload les colonnes a utilisé
    timestamp = row['Global_Time']
    ego_lane = row['Lane_ID']
    ego_y = row['Local_Y']
    ego_x = row['Local_X']
    ego_length = row['v_Length']
    ego_width = row['v_Width']
    
    #Calcule les distances avec Local_X et Local_Y
    lat_dist1 = (data['Local_X']-ego_x)*0.305
    lat_dist2 = (ego_x-data['Local_X'])*0.305
    long_dist = abs(ego_y-data['Local_Y'])*0.305
    r_Lane_cond = (data['Global_Time']==timestamp) & (lat_dist1<=max_width) & (long_dist<=max_dist)
    l_Lane_cond = (data['Global_Time']==timestamp) & (lat_dist2<=max_width) & (long_dist<=max_dist)
    
    #Remplire les différents voisins en fonction d'un tri 
    r_vehicles = data[r_Lane_cond]
    r_vehicles.sort_values(by=['Local_Y'], inplace=True)
    l_vehicles = data[l_Lane_cond]
    l_vehicles.sort_values(by=['Local_Y'], inplace=True)
    
    try:
        
        r_vehicles_id = r_vehicles['Vehicle_ID'].values
        r_vehicles_X = r_vehicles['Local_X'].values
        r_vehicles_Y = r_vehicles['Local_Y'].values
        r_vehicles_vel = r_vehicles['v_Vel'].values
        r_vehicles_acc = r_vehicles['v_Acc'].values
        r_vehicles_lane = r_vehicles['Lane_ID'].values
    except:
        r_v_id = 0
        r_vehicles_X = 0
        r_vehicles_Y = 0
        r_vehicles_vel = 0
        r_vehicles_acc = 0
        r_vehicles_lane = 0
    try:
        l_vehicles_id = l_vehicles['Vehicle_ID'].values
        l_vehicles_X = l_vehicles['Local_X'].values
        l_vehicles_Y = l_vehicles['Local_Y'].values
        l_vehicles_vel = l_vehicles['v_Vel'].values
        l_vehicles_acc = l_vehicles['v_Acc'].values
        l_vehicles_lane = l_vehicles['Lane_ID'].values
    except:
        l_v_id = 0
        l_vehicles_X = 0
        l_vehicles_Y = 0
        l_vehicles_vel = 0
        l_vehicles_acc = 0
        l_vehicles_lane=0
    
    return pd.Series([r_vehicles_id ,r_vehicles_X, r_vehicles_Y,r_vehicles_lane, r_vehicles_vel, r_vehicles_acc, l_vehicles_id, 
                      l_vehicles_X, l_vehicles_Y, l_vehicles_lane,l_vehicles_vel, l_vehicles_acc])

def prepare_data(inputfile, output, outputdir):
    pd.set_option('mode.chained_assignment', None)
    data = pd.read_csv(inputfile)
    vehicle_ids = data.groupby('Vehicle_ID').groups.keys()
    new_df = pd.DataFrame()
    progress = 0
    with tqdm(total=len(vehicle_ids)) as pbar:
        for v_id in vehicle_ids:
                v_df = data[data.Vehicle_ID==v_id]
                others = data[data.Vehicle_ID!=v_id]
                v_df[['Right_v_IDs','Right_v_X','Right_v_Y','Right_v_Lanes','Right_v_Vels','Right_v_Acc','Left_v_IDs','Left_v_X',
                      'Left_v_Y','Left_v_Lanes', 'Left_v_Vels','Left_v_Acc']] = v_df.parallel_apply(get_neighbors,axis=1,args=(others,))
            
                    
                new_df = new_df.append(v_df)
                new_df.to_csv(output,index=False)
                progress = progress+1
                sys.stdout.write("\r%d of %d vehicles" % (progress, len(vehicle_ids)))
                sys.stdout.flush()
                pbar.update(1)
    return new_df
#Data Originale
filename1='Data/NGSIM_.csv'

#le repértoire ou mettre mon nouveau dataset préparé
outputdir1='DataSet/Prepared'

#fichier csv contenant ce dataset
outputfile1='DataSet/Prepared/trajectories-prepared.csv' # trajectories-prepared.csv

prepare_data(filename1,outputfile1,outputdir1)

  0%|                                                                                         | 0/1725 [00:00<?, ?it/s]

1 of 1725 vehicles

  0%|                                                                              | 1/1725 [01:24<40:26:30, 84.45s/it]

2 of 1725 vehicles

  0%|                                                                              | 2/1725 [02:26<34:04:31, 71.20s/it]

## En raison que la preparation de donnes prende beaucoup de temp,  j'ai utilisé un dataset deja préparé

In [3]:
data=pd.read_csv('Data/prepared/trajectories-prepared.csv')
data

,Vehicle_ID,Frame_ID,Total_Frames,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_Length,v_Width,...,Right_v_Y,Right_v_Lanes,Right_v_Vels,Right_v_Acc,Left_v_IDs,Left_v_X,Left_v_Y,Left_v_Lanes,Left_v_Vels,Left_v_Acc
0,1,12,884,1113433136100,16.884,48.213,6042842.116,2133117.662,14.3,6.4,...,[84.274],[3],[11.16],[10.67],[36],[27.492],[84.274],[3],[11.16],[10.67]
1,1,13,884,1113433136200,16.938,49.463,6042842.012,2133118.909,14.3,6.4,...,[85.465],[3],[11.99],[6.76],[36],[27.433],[85.465],[3],[11.99],[6.76]
2,1,14,884,1113433136300,16.991,50.712,6042841.908,2133120.155,14.3,6.4,...,[86.736],[3],[12.45],[1.49],[36],[27.423],[86.736],[3],[12.45],[1.49]
3,1,15,884,1113433136400,17.045,51.963,6042841.805,2133121.402,14.3,6.4,...,[88.003],[3],[12.53],[-0.3],[36],[27.434],[88.003],[3],[12.53],[-0.3]
4,1,16,884,1113433136500,17.098,53.213,6042841.701,2133122.649,14.3,6.4,...,[89.254],[3],[12.51],[-0.16],[36],[27.434],[89.254],[3],[12.51],[-0.16]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90851,243,1145,373,1113433249400,4.459,502.396,6042773.961,2133566.836,15.3,7.9,...,[278.069 287.038 353.889 369.654 422.222 435.3...,[2 1 1 2 2 1 2 2 1 2 2 1 2 2 1],[ 4.88 37.96 40. 22.66 17.31 41.16 21.39 30....,[ 11.2 0.14 0. -11.2 10.02 -6.57 -0...,[329 263 318 287 280 299 335 269 270 256 336 2...,[15.77 4.884 67.222 54.796 40.575 30.266 87....,[278.069 287.038 289.631 295.93 299.506 310.9...,[2 1 6 5 4 3 7 4 5 1 6 3 2 4 2 5 1 6 3 2 4 6 3...,[ 4.88 37.96 24.97 17.6 33.05 21.73 30.37 30....,[ 11.2 0.14 0.05 -11.2 -7.31 3.95 0...
90852,243,1146,373,1113433249500,4.458,505.931,6042773.528,2133570.338,15.3,7.9,...,[278.719 290.83 357.889 371.847 424.039 439.4...,[2 1 1 2 2 1 2 2 1 2 2 1 2 2 1],[ 7.34 38. 40. 21.68 17.97 40.6 21.37 30....,[11.2 0.69 0. -8.27 3.78 -6.06 -0.08 0....,[329 263 318 287 280 299 335 269 270 336 256 2...,[16.054 4.866 67.36 54.668 40.575 30.197 87....,[278.719 290.83 292.124 297.607 302.777 313.1...,[2 1 6 5 4 3 7 4 5 6 1 3 2 4 2 5 1 6 3 2 4 6 3...,[ 7.34 38. 25. 16.68 31.93 21.8 30.37 29....,[ 11.2 0.69 0.53 -6.91 -11.2 -2.68 0...
90853,243,1147,373,1113433249600,4.457,509.423,6042773.103,2133573.782,15.3,7.9,...,[294.639 361.893 373.955 425.916 443.455 467.6...,[1 1 2 2 1 2 2 1 2 2 1 2 2 1],[38.03 39.97 20.95 17.97 40.06 21.37 29.99 32....,[-0.04 -0.52 -5.54 -4.65 -3.98 0. -0.35 -1....,[318 263 287 280 299 335 269 270 336 289 256 2...,[67.499 4.848 54.463 40.527 30.098 87.312 40....,[294.628 294.639 299.19 305.889 315.374 342.1...,[6 1 5 4 3 7 4 5 6 3 1 2 4 2 5 1 6 3 2 4 6 3 5...,[25.04 38.03 16.39 30.41 21.4 30.37 29.71 21....,[ 0.34 -0.04 0.92 -11.2 -6.55 0. 2...
90854,243,1148,373,1113433249700,4.456,512.905,6042772.674,2133577.256,15.3,7.9,...,[298.458 365.882 376.025 427.701 447.439 469.8...,[1 1 2 2 1 2 2 1 2 2 1 2 2 1],[37.87 39.96 20.31 17.48 39.59 21.37 29.83 32....,[-2.59 0.08 -6.39 -6.18 -4.42 0.03 -2.51 2....,[318 263 287 280 299 335 269 270 336 289 256 2...,[67.637 4.83 54.34 40.45 29.993 87.319 40....,[297.142 298.458 300.81 308.874 317.473 345.1...,[6 1 5 4 3 7 4 5 6 3 1 2 4 2 5 6 1 3 2 4 6 3 5...,[25. 37.87 16.54 29.07 20.88 30.37 29.89 20....,[ -0.83 -2.59 3.94 -10.49 -4.74 0. 1...


# Trouver le vehicule proche

In [4]:
def find_nearest_target(df):
    total_rows = len(df)
    progress_bar = tqdm(total=total_rows, desc='Processing rows')
    for i, row in df.iterrows():
        x = row["Local_X"]
        
        right_ids = df.at[i, "Right_v_IDs"]
        right_ids = [float(num) for num in re.findall(r"[-+]?\d*\.\d+|\d+", str(right_ids))] 
        
        left_ids = df.at[i, "Left_v_IDs"]
        left_ids = [float(num) for num in re.findall(r"[-+]?\d*\.\d+|\d+", str(left_ids))] 
        
        right_x = df.at[i, "Right_v_X"]
        right_x = np.array([float(num) for num in re.findall(r"[-+]?\d*\.\d+|\d+", str(right_x))]) 
        
        left_x = df.at[i, "Left_v_X"]
        left_x = np.array([float(num) for num in re.findall(r"[-+]?\d*\.\d+|\d+", str(left_x))]) 
        
        distances = []
        if len(right_x) > 0:
            #Calculer la distance enntre x et le local x du véhicule de droite
            distance = np.abs(x - right_x)
            # Ajouter cette distance dans une liste
            distances.extend(distance)
        if len(left_x) > 0:
            #Calculer la distance enntre x et le local x du véhicule de gauche
            distance = np.abs(x - left_x)
            #Ajouter cette distance dans une liste
            distances.extend(distance)
        
        if len(distances) > 0:
            # Trouver l'index de la plus petite distance et prendre le vehicule proche
            min_index = np.argmin(distances)
            if min_index < len(right_x):
                target_id = right_ids[min_index]
            else:
                min_index -= len(right_x)
                target_id = left_ids[min_index]
        else:
            target_id = None
        # Ajout du target a notre Dataframe
        df.at[i, "Vehicule_Target_ID"] = target_id
        progress_bar.update(1)
    progress_bar.close()
    return df


In [5]:
data = find_nearest_target(data)
data

Processing rows: 100%|██████████████████████████████████████████████████████████| 90856/90856 [01:47<00:00, 843.41it/s]


,Vehicle_ID,Frame_ID,Total_Frames,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_Length,v_Width,...,Right_v_Lanes,Right_v_Vels,Right_v_Acc,Left_v_IDs,Left_v_X,Left_v_Y,Left_v_Lanes,Left_v_Vels,Left_v_Acc,Vehicule_Target_ID
0,1,12,884,1113433136100,16.884,48.213,6042842.116,2133117.662,14.3,6.4,...,[3],[11.16],[10.67],[36],[27.492],[84.274],[3],[11.16],[10.67],36.0
1,1,13,884,1113433136200,16.938,49.463,6042842.012,2133118.909,14.3,6.4,...,[3],[11.99],[6.76],[36],[27.433],[85.465],[3],[11.99],[6.76],36.0
2,1,14,884,1113433136300,16.991,50.712,6042841.908,2133120.155,14.3,6.4,...,[3],[12.45],[1.49],[36],[27.423],[86.736],[3],[12.45],[1.49],36.0
3,1,15,884,1113433136400,17.045,51.963,6042841.805,2133121.402,14.3,6.4,...,[3],[12.53],[-0.3],[36],[27.434],[88.003],[3],[12.53],[-0.3],36.0
4,1,16,884,1113433136500,17.098,53.213,6042841.701,2133122.649,14.3,6.4,...,[3],[12.51],[-0.16],[36],[27.434],[89.254],[3],[12.51],[-0.16],36.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90851,243,1145,373,1113433249400,4.459,502.396,6042773.961,2133566.836,15.3,7.9,...,[2 1 1 2 2 1 2 2 1 2 2 1 2 2 1],[ 4.88 37.96 40. 22.66 17.31 41.16 21.39 30....,[ 11.2 0.14 0. -11.2 10.02 -6.57 -0...,[329 263 318 287 280 299 335 269 270 256 336 2...,[15.77 4.884 67.222 54.796 40.575 30.266 87....,[278.069 287.038 289.631 295.93 299.506 310.9...,[2 1 6 5 4 3 7 4 5 1 6 3 2 4 2 5 1 6 3 2 4 6 3...,[ 4.88 37.96 24.97 17.6 33.05 21.73 30.37 30....,[ 11.2 0.14 0.05 -11.2 -7.31 3.95 0...,256.0
90852,243,1146,373,1113433249500,4.458,505.931,6042773.528,2133570.338,15.3,7.9,...,[2 1 1 2 2 1 2 2 1 2 2 1 2 2 1],[ 7.34 38. 40. 21.68 17.97 40.6 21.37 30....,[11.2 0.69 0. -8.27 3.78 -6.06 -0.08 0....,[329 263 318 287 280 299 335 269 270 336 256 2...,[16.054 4.866 67.36 54.668 40.575 30.197 87....,[278.719 290.83 292.124 297.607 302.777 313.1...,[2 1 6 5 4 3 7 4 5 6 1 3 2 4 2 5 1 6 3 2 4 6 3...,[ 7.34 38. 25. 16.68 31.93 21.8 30.37 29....,[ 11.2 0.69 0.53 -6.91 -11.2 -2.68 0...,256.0
90853,243,1147,373,1113433249600,4.457,509.423,6042773.103,2133573.782,15.3,7.9,...,[1 1 2 2 1 2 2 1 2 2 1 2 2 1],[38.03 39.97 20.95 17.97 40.06 21.37 29.99 32....,[-0.04 -0.52 -5.54 -4.65 -3.98 0. -0.35 -1....,[318 263 287 280 299 335 269 270 336 289 256 2...,[67.499 4.848 54.463 40.527 30.098 87.312 40....,[294.628 294.639 299.19 305.889 315.374 342.1...,[6 1 5 4 3 7 4 5 6 3 1 2 4 2 5 1 6 3 2 4 6 3 5...,[25.04 38.03 16.39 30.41 21.4 30.37 29.71 21....,[ 0.34 -0.04 0.92 -11.2 -6.55 0. 2...,263.0
90854,243,1148,373,1113433249700,4.456,512.905,6042772.674,2133577.256,15.3,7.9,...,[1 1 2 2 1 2 2 1 2 2 1 2 2 1],[37.87 39.96 20.31 17.48 39.59 21.37 29.83 32....,[-2.59 0.08 -6.39 -6.18 -4.42 0.03 -2.51 2....,[318 263 287 280 299 335 269 270 336 289 256 2...,[67.637 4.83 54.34 40.45 29.993 87.319 40....,[297.142 298.458 300.81 308.874 317.473 345.1...,[6 1 5 4 3 7 4 5 6 3 1 2 4 2 5 6 1 3 2 4 6 3 5...,[25. 37.87 16.54 29.07 20.88 30.37 29.89 20....,[ -0.83 -2.59 3.94 -10.49 -4.74 0. 1...,263.0


# Detection de changement de voie

In [6]:
def detect_lane_changes(df):
    # Ajout d'une nouvelle colonne pour stocker l'information sur les changements de voie
    df['Lane_Change'] = 'Non_Cutin'
    total_rows = len(df)
    progress_bar = tqdm(total=total_rows, desc='Processing rows')

    # Itération à travers chaque ligne du dataframe
    for i in range(1, len(df)):
        # Vérification si le Lane_ID change par rapport à la ligne précédente
        if df.loc[i, 'Lane_ID'] != df.loc[max(i-7,0), 'Lane_ID']:
            # Sélection de la séquence de 7 véhicules qui précèdent la ligne actuelle
            preceding_vehicles = df.iloc[i-7:i]
            # Vérification si la séquence contient des valeurs manquantes
            if preceding_vehicles.isna().sum().sum() == 0:
                # Calcul de la distance entre le véhicule en cours et le précédent
                space_headway = df.loc[i, 'Space_Headway']
                # Calcul du temps entre le véhicule en cours et le précédent
                time_headway = df.loc[i, 'Time_Headway']
                # Vérification si la distance et le temps entre les deux véhicules sont suffisants
                if space_headway > preceding_vehicles['Space_Headway'].min() and time_headway > 1.5:
                    # Calcul de la différence de vitesse entre les deux véhicules
                    speed_difference = preceding_vehicles['v_Vel'].max() - df.loc[i, 'v_Vel']
                    # Vérification si la différence de vitesse est suffisamment grande
                    if speed_difference > 0:
                        # Calcul de l'accélération du véhicule en cours
                        acceleration = df.loc[i, 'v_Acc']
                        # Vérification si l'accélération est suffisamment grande
                        if acceleration > 5:
                            # Marquage du changement de voie comme "Non_Cutin"
                            df.loc[i, 'Lane_Change'] = 'NoCutin'
                        else:
                            # Marquage du changement de voie comme "Cutin"
                            df.loc[i, 'Lane_Change'] = 'Cutin'
                    else:
                        # Marquage du changement de voie comme "Non_Cutin"
                        df.loc[i, 'Lane_Change'] = 'NoCutin'
                else:
                    # Ignore la ligne actuelle si la séquence contient des valeurs manquantes
                    pass
            else:
                # Ignore la ligne actuelle si la séquence contient des valeurs manquantes
                pass
        progress_bar.update(1)
    progress_bar.close()
    return df

In [7]:
data = detect_lane_changes(data)
data

Processing rows: 100%|████████████████████████████████████████████████████████▉| 90855/90856 [00:12<00:00, 7392.51it/s]


,Vehicle_ID,Frame_ID,Total_Frames,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_Length,v_Width,...,Right_v_Vels,Right_v_Acc,Left_v_IDs,Left_v_X,Left_v_Y,Left_v_Lanes,Left_v_Vels,Left_v_Acc,Vehicule_Target_ID,Lane_Change
0,1,12,884,1113433136100,16.884,48.213,6042842.116,2133117.662,14.3,6.4,...,[11.16],[10.67],[36],[27.492],[84.274],[3],[11.16],[10.67],36.0,Non_Cutin
1,1,13,884,1113433136200,16.938,49.463,6042842.012,2133118.909,14.3,6.4,...,[11.99],[6.76],[36],[27.433],[85.465],[3],[11.99],[6.76],36.0,Non_Cutin
2,1,14,884,1113433136300,16.991,50.712,6042841.908,2133120.155,14.3,6.4,...,[12.45],[1.49],[36],[27.423],[86.736],[3],[12.45],[1.49],36.0,Non_Cutin
3,1,15,884,1113433136400,17.045,51.963,6042841.805,2133121.402,14.3,6.4,...,[12.53],[-0.3],[36],[27.434],[88.003],[3],[12.53],[-0.3],36.0,Non_Cutin
4,1,16,884,1113433136500,17.098,53.213,6042841.701,2133122.649,14.3,6.4,...,[12.51],[-0.16],[36],[27.434],[89.254],[3],[12.51],[-0.16],36.0,Non_Cutin
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90851,243,1145,373,1113433249400,4.459,502.396,6042773.961,2133566.836,15.3,7.9,...,[ 4.88 37.96 40. 22.66 17.31 41.16 21.39 30....,[ 11.2 0.14 0. -11.2 10.02 -6.57 -0...,[329 263 318 287 280 299 335 269 270 256 336 2...,[15.77 4.884 67.222 54.796 40.575 30.266 87....,[278.069 287.038 289.631 295.93 299.506 310.9...,[2 1 6 5 4 3 7 4 5 1 6 3 2 4 2 5 1 6 3 2 4 6 3...,[ 4.88 37.96 24.97 17.6 33.05 21.73 30.37 30....,[ 11.2 0.14 0.05 -11.2 -7.31 3.95 0...,256.0,Non_Cutin
90852,243,1146,373,1113433249500,4.458,505.931,6042773.528,2133570.338,15.3,7.9,...,[ 7.34 38. 40. 21.68 17.97 40.6 21.37 30....,[11.2 0.69 0. -8.27 3.78 -6.06 -0.08 0....,[329 263 318 287 280 299 335 269 270 336 256 2...,[16.054 4.866 67.36 54.668 40.575 30.197 87....,[278.719 290.83 292.124 297.607 302.777 313.1...,[2 1 6 5 4 3 7 4 5 6 1 3 2 4 2 5 1 6 3 2 4 6 3...,[ 7.34 38. 25. 16.68 31.93 21.8 30.37 29....,[ 11.2 0.69 0.53 -6.91 -11.2 -2.68 0...,256.0,Non_Cutin
90853,243,1147,373,1113433249600,4.457,509.423,6042773.103,2133573.782,15.3,7.9,...,[38.03 39.97 20.95 17.97 40.06 21.37 29.99 32....,[-0.04 -0.52 -5.54 -4.65 -3.98 0. -0.35 -1....,[318 263 287 280 299 335 269 270 336 289 256 2...,[67.499 4.848 54.463 40.527 30.098 87.312 40....,[294.628 294.639 299.19 305.889 315.374 342.1...,[6 1 5 4 3 7 4 5 6 3 1 2 4 2 5 1 6 3 2 4 6 3 5...,[25.04 38.03 16.39 30.41 21.4 30.37 29.71 21....,[ 0.34 -0.04 0.92 -11.2 -6.55 0. 2...,263.0,Non_Cutin
90854,243,1148,373,1113433249700,4.456,512.905,6042772.674,2133577.256,15.3,7.9,...,[37.87 39.96 20.31 17.48 39.59 21.37 29.83 32....,[-2.59 0.08 -6.39 -6.18 -4.42 0.03 -2.51 2....,[318 263 287 280 299 335 269 270 336 289 256 2...,[67.637 4.83 54.34 40.45 29.993 87.319 40....,[297.142 298.458 300.81 308.874 317.473 345.1...,[6 1 5 4 3 7 4 5 6 3 1 2 4 2 5 6 1 3 2 4 6 3 5...,[25. 37.87 16.54 29.07 20.88 30.37 29.89 20....,[ -0.83 -2.59 3.94 -10.49 -4.74 0. 1...,263.0,Non_Cutin


## Cutin : 1
## Non_Cutin : 0

In [8]:
data["Lane_Change"] = data["Lane_Change"].apply(lambda x: 1 if x == "Cutin" else 0)
data

,Vehicle_ID,Frame_ID,Total_Frames,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_Length,v_Width,...,Right_v_Vels,Right_v_Acc,Left_v_IDs,Left_v_X,Left_v_Y,Left_v_Lanes,Left_v_Vels,Left_v_Acc,Vehicule_Target_ID,Lane_Change
0,1,12,884,1113433136100,16.884,48.213,6042842.116,2133117.662,14.3,6.4,...,[11.16],[10.67],[36],[27.492],[84.274],[3],[11.16],[10.67],36.0,0
1,1,13,884,1113433136200,16.938,49.463,6042842.012,2133118.909,14.3,6.4,...,[11.99],[6.76],[36],[27.433],[85.465],[3],[11.99],[6.76],36.0,0
2,1,14,884,1113433136300,16.991,50.712,6042841.908,2133120.155,14.3,6.4,...,[12.45],[1.49],[36],[27.423],[86.736],[3],[12.45],[1.49],36.0,0
3,1,15,884,1113433136400,17.045,51.963,6042841.805,2133121.402,14.3,6.4,...,[12.53],[-0.3],[36],[27.434],[88.003],[3],[12.53],[-0.3],36.0,0
4,1,16,884,1113433136500,17.098,53.213,6042841.701,2133122.649,14.3,6.4,...,[12.51],[-0.16],[36],[27.434],[89.254],[3],[12.51],[-0.16],36.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90851,243,1145,373,1113433249400,4.459,502.396,6042773.961,2133566.836,15.3,7.9,...,[ 4.88 37.96 40. 22.66 17.31 41.16 21.39 30....,[ 11.2 0.14 0. -11.2 10.02 -6.57 -0...,[329 263 318 287 280 299 335 269 270 256 336 2...,[15.77 4.884 67.222 54.796 40.575 30.266 87....,[278.069 287.038 289.631 295.93 299.506 310.9...,[2 1 6 5 4 3 7 4 5 1 6 3 2 4 2 5 1 6 3 2 4 6 3...,[ 4.88 37.96 24.97 17.6 33.05 21.73 30.37 30....,[ 11.2 0.14 0.05 -11.2 -7.31 3.95 0...,256.0,0
90852,243,1146,373,1113433249500,4.458,505.931,6042773.528,2133570.338,15.3,7.9,...,[ 7.34 38. 40. 21.68 17.97 40.6 21.37 30....,[11.2 0.69 0. -8.27 3.78 -6.06 -0.08 0....,[329 263 318 287 280 299 335 269 270 336 256 2...,[16.054 4.866 67.36 54.668 40.575 30.197 87....,[278.719 290.83 292.124 297.607 302.777 313.1...,[2 1 6 5 4 3 7 4 5 6 1 3 2 4 2 5 1 6 3 2 4 6 3...,[ 7.34 38. 25. 16.68 31.93 21.8 30.37 29....,[ 11.2 0.69 0.53 -6.91 -11.2 -2.68 0...,256.0,0
90853,243,1147,373,1113433249600,4.457,509.423,6042773.103,2133573.782,15.3,7.9,...,[38.03 39.97 20.95 17.97 40.06 21.37 29.99 32....,[-0.04 -0.52 -5.54 -4.65 -3.98 0. -0.35 -1....,[318 263 287 280 299 335 269 270 336 289 256 2...,[67.499 4.848 54.463 40.527 30.098 87.312 40....,[294.628 294.639 299.19 305.889 315.374 342.1...,[6 1 5 4 3 7 4 5 6 3 1 2 4 2 5 1 6 3 2 4 6 3 5...,[25.04 38.03 16.39 30.41 21.4 30.37 29.71 21....,[ 0.34 -0.04 0.92 -11.2 -6.55 0. 2...,263.0,0
90854,243,1148,373,1113433249700,4.456,512.905,6042772.674,2133577.256,15.3,7.9,...,[37.87 39.96 20.31 17.48 39.59 21.37 29.83 32....,[-2.59 0.08 -6.39 -6.18 -4.42 0.03 -2.51 2....,[318 263 287 280 299 335 269 270 336 289 256 2...,[67.637 4.83 54.34 40.45 29.993 87.319 40....,[297.142 298.458 300.81 308.874 317.473 345.1...,[6 1 5 4 3 7 4 5 6 3 1 2 4 2 5 6 1 3 2 4 6 3 5...,[25. 37.87 16.54 29.07 20.88 30.37 29.89 20....,[ -0.83 -2.59 3.94 -10.49 -4.74 0. 1...,263.0,0


## Nombre de Cutins et de Non_Cutins dans notre DataSet

In [9]:
cutin_count = (data["Lane_Change"] == 1).sum()
print("Nombre de Cutin :", cutin_count)

nocutin_count = (data["Lane_Change"] == 0).sum()
print("Nombre de NoCutin :", nocutin_count)

Nombre de Cutin : 756
Nombre de NoCutin : 90100


# Creation de Model

### donnees de train et donnee de test

In [10]:
from sklearn.model_selection import train_test_split

X=data.drop(['Vehicle_ID','Frame_ID','Total_Frames','Global_Time','Right_v_IDs','Right_v_X','Right_v_Y','Right_v_Lanes','Right_v_Vels','Right_v_Acc','Left_v_IDs','Left_v_X','Left_v_Y','Left_v_Lanes','Left_v_Vels','Left_v_Acc','Vehicule_Target_ID','Lane_Change'],axis=1)
Y=data['Lane_Change']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

# Random forest

In [11]:
from sklearn.ensemble import RandomForestClassifier

# création d'un modèle de classification Random Forest avec 100 arbres
rf = RandomForestClassifier(n_estimators=100)

# entraînement du modèle sur les données d'entraînement
rf.fit(X_train, y_train)

# prédiction des étiquettes de classe pour les données de test
y_pred = rf.predict(X_test)

# évaluation des performances du modèle en comparant les étiquettes de classe prédites avec les étiquettes de classe réelles
accuracy = rf.score(X_test, y_test)
#============================
print("evaluation : accuracy : {:.2f}%".format(accuracy * 100))

#============================
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# afficher les résultats
print("Recall:", recall*100, '%')
print("Precision:", precision*100, '%')
print("F1-score:", f1*100, '%')
#============================
print("accuracy : {:.2f}%".format(accuracy * 100))


evaluation : accuracy : 99.77%
Recall: 72.5925925925926 %
Precision: 95.14563106796116 %
F1-score: 82.3529411764706 %
accuracy : 99.77%
